In [91]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chatbots-intent-recognition-dataset/Intent.json


In [178]:
import json
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance

#load data from json
with open('/kaggle/input/chatbots-intent-recognition-dataset/Intent.json') as json_file:
    data = json.load(json_file)
    
data = data['intents']

In [179]:
df = pd.DataFrame(columns=['intent', 'text', 'response'])

for d in data:
    intent = d['intent']
    
    for text_, res_ in zip(d['text'], d['responses']):
        row = {"intent": intent, "text": text_, "response":res_}
        
        df= df.append(row, ignore_index=True)
        
df.head()

,intent,text,response
0,Greeting,Hi,"Hi human, please tell me your GeniSys user"
1,Greeting,Hi there,"Hello human, please tell me your GeniSys user"
2,Greeting,Hola,"Hola human, please tell me your GeniSys user"
3,GreetingResponse,My user is Adam,Great! Hi <HUMAN>! How can I help?
4,GreetingResponse,This is Adam,"Good! Hi <HUMAN>, how can I help you?"


In [172]:
#total intent in our dataset
df.intent.unique()

array(['Greeting', 'GreetingResponse', 'CourtesyGreeting',
       'CourtesyGreetingResponse', 'CurrentHumanQuery', 'NameQuery',
       'RealNameQuery', 'TimeQuery', 'Thanks', 'NotTalking2U',
       'UnderstandQuery', 'Shutup', 'Swearing', 'GoodBye',
       'CourtesyGoodBye', 'WhoAmI', 'Clever', 'Gossip', 'Jokes',
       'PodBayDoor', 'PodBayDoorResponse', 'SelfAware'], dtype=object)

## **Data Preprocessing**

In [173]:
import re
import nltk
nltk.download()

from nltk.corpus import stopwords

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


...

Hit Enter to continue:  l


  [*] conll2002........... CONLL 2002 Named Entity Recognition Corpus
  [*] conll2007........... Dependency Treebanks from CoNLL 2007 (Catalan
                           and Basque Subset)
  [*] crubadan............ Crubadan Corpus
  [*] dependency_treebank. Dependency Parsed Treebank
  [ ] dolch............... Dolch Word List
  [*] europarl_raw........ Sample European Parliament Proceedings Parallel
                           Corpus
  [ ] extended_omw........ Extended Open Multilingual WordNet
  [*] floresta............ Portuguese Treebank
  [ ] framenet_v15........ FrameNet 1.5
  [ ] framenet_v17........ FrameNet 1.7
  [*] gazetteers.......... Gazeteer Lists
  [*] genesis............. Genesis Corpus
  [*] gutenberg........... Project Gutenberg Selections
  [*] ieer................ NIST IE-ER DATA SAMPLE
  [*] inaugural........... C-Span Inaugural Address Corpus
  [*] indian.............. Indian Language POS-Tagged Corpus
  [*] jeita............... JEITA Public Morphologically Tagged 

Hit Enter to continue:  q



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------


Downloader>  q


In [180]:
def text_preprocessing(text):
    """
    converting raw text and response into a string of words. Remove unnecessay data.
    Input: Raw string input
    output: return clean string
    """
    #removing extra punctuations, numbers and etc
    text_only = re.sub(r'[^a-zA-z.?!\']', " ", text)
    text_only = re.sub(r'[ ]+', ' ', text_only)
    
    #convert word into lowercase and split
    words = text.lower().split()
    return(" ".join(words))


In [181]:
text = df['text'].apply(text_preprocessing)
resp = df['response'].apply(text_preprocessing)

In [182]:
df['text'] = text
df['response'] = resp
df.head()

,intent,text,response
0,Greeting,hi,"hi human, please tell me your genisys user"
1,Greeting,hi there,"hello human, please tell me your genisys user"
2,Greeting,hola,"hola human, please tell me your genisys user"
3,GreetingResponse,my user is adam,great! hi <human>! how can i help?
4,GreetingResponse,this is adam,"good! hi <human>, how can i help you?"


In [183]:
def check_simialarity(text1, text2):
    sentences = [text1, text2]
    
    #creating a vocabulary with sentences
    #Bags of Words
    count_vec = CountVectorizer(analyzer = "word", max_features=1000)
    sentences_vec = count_vec.fit_transform(sentences)
    
    vocab = count_vec.get_feature_names()
    
    #converting into an array of list
    text1_vec = sentences_vec.toarray()[0].tolist()
    text2_vec = sentences_vec.toarray()[1].tolist()
    
    dist = distance.cosine(text1_vec, text2_vec)
    
    #calculate the score from distance b/w 0~1
    return round((1-dist), 2)

In [184]:
def respond(text):
    maximum = float("-inf")
    reponse, closet = "", ""
    
    for data in df.iterrows():
        #get the text score which is similar to input
        similarity = check_simialarity(text, data[1]['text'])
        
        #check the similarity, similarity between 0 to 1
        if similarity > maximum:
            maximum = similarity
            response = data[1]['response']
            closest = data[1]['text']
        
    return response

In [188]:
print("Hey! How are you.Let's Chat")
while True:
    text = str(input("\nInput: "))
    if text.lower() == "q":
        print("Response: Bot is going to sleep...")
        break
    print("Response:",respond(text))

Hey! How are you.Let's Chat



Input:  what is your name


Response: you can call me geni



Input:  what are you doing


Response: hi, how are you? i am great thanks! please tell me your genisys user


...

Response: so i went down the local supermarket, i said, 'i want to make a complaint, this vinegar's got lumps in it', he said, 'those are pickled onions'.



Input:  give me a tip of selfaware


Response: so i went down the local supermarket, i said, 'i want to make a complaint, this vinegar's got lumps in it', he said, 'those are pickled onions'.



Input:  q


Response: Bot is going to sleep...


## Trying with other way

In [167]:
#creating a vocabulary with sentences
#Bags of Words
count_vec = CountVectorizer(analyzer = "word", max_features=1000)
sentences_vec = count_vec.fit_transform(df['text'])
response_vec = count_vec.fit_transform(df['response'])

vocab = count_vec.get_feature_names()
#     print(vocab)

sentences_vec = sentences_vec.toarray()
response_vec = response_vec.toarray()

sentences_vec = tf.keras.preprocessing.sequence.pad_sequences(sentences_vec, maxlen=219, padding='pre')
response_vec = tf.keras.preprocessing.sequence.pad_sequences(response_vec, maxlen=219, padding='pre')

from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 1000) 

# Fit the forest to the training set, using the bag of words as 
forest = forest.fit(sentences_vec, response_vec)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [163]:
text = [input()]

 tell me a joke about anything


In [168]:
import tensorflow as tf
# transforming the input and convert to a numpy array
test_data_features = count_vec.transform(text)

test_data_features = test_data_features.toarray()

  (0, 120)	1
  (0, 182)	1


In [169]:
result = forest.predict(test_data_features)
result

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)